The following is an incomplete analysis of the distribution of deep-sea sponges based on ocean sediment. It is incomplete due to not leaving enough time for the code which matches each sponge to its sediment to run, with an estimated run time of multiple days.

In [ ]:
#Imports for statistical analysis and database management
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

#Import needed to open sediment data
import netCDF4 as nc4

#import for testing code run time
import time

The data used here comes from the NOAA Deep Sea Coral and Sponge Map (https://www.ncei.noaa.gov/products/noaa-deep-sea-coral-and-sponge-map).

In [4]:
df=pd.read_csv('deep_sea_coral_sponge_data.csv')

C:\Users\Sam She\AppData\Local\Temp\ipykernel_11508\2917557790.py:1: DtypeWarning: Columns (5,13,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('deep_sea_coral_sponge_data.csv')


In [5]:
#create dataframe of just sponges
target_sponge_name_list=['demosponge','glass sponge','homoscleromorph sponge','calcareous sponge'] #list of higher sponge clades present, as per vernacular names

sponge_df=df[df.VernacularNameCategory=='sponge (unspecified)']
for name in target_sponge_name_list:
    sponge_df=pd.concat([sponge_df,df[df.VernacularNameCategory==name]])

demosponge
glass sponge
homoscleromorph sponge
calcareous sponge


In [6]:
#seperate sponges based on higher clade
uncat_df=df[df.VernacularNameCategory=='sponge (unspecified)']
demo_df=df[df.VernacularNameCategory=='demosponge']
glass_df=df[df.VernacularNameCategory=='glass sponge']
homo_df=df[df.VernacularNameCategory=='homoscleromorph sponge']
calc_df=df[df.VernacularNameCategory=='calcareous sponge']

The data used here comes from the GPlatesPortal Seafloor Lithology Map (https://portal.gplates.org/portal/seafloor/).

In [7]:
ds = nc4.Dataset('seabed_lithology_v1.nc')

#Code needed to prepare dataset for use.
lon=ds.variables['lon'][:]
lat=ds.variables['lat'][:]
z=ds.variables['z'][:,:]

ds.close()

dict_keys(['lon', 'lat', 'z'])


In [8]:
#Dictionaries to match z value of lithology dataset to sediment name.
lithology_key={0:'No Data',
               1:'Gravel and coarser',2:'Sand',3:'Silt',4:'Clay',
               5:'Calcareous Ooze',6:'Radiolarian Ooze',7:'Diatom Ooze',
               8:'Sponge Spicules',9:'Mixed Calcareous-Siliceous Ooze',
               10:'Shells and Coral Fragments',
               11:'Ash and Volcanic Sand/Gravel',
               12:'Fine-Grained Calcareous Sediment',13:'Siliceous Mud'}
lithology_type_key={'No Data':'No Data',
                    'Gravel and coarser':'Siliciclastic','Sand':'Siliciclastic','Silt':'Siliciclastic','Clay':'Siliciclastic',
                    'Calcareous Ooze':'Biogenic','Radiolarian Ooze':'Biogenic','Diatom Ooze':'Biogenic',
                    'Sponge Spicules':'Biogenic','Mixed Calcareous-Siliceous Ooze':'Biogenic',
                    'Shells and Coral Fragments':'Biogenic',
                    'Ash and Volcanic Sand/Gravel':'Volcaniclastic',
                    'Fine-Grained Calcareous Sediment':'Transitional','Siliceous Mud':'Transitional'}

In [9]:
#Function to calculate distance between 2 points on a globe. Provided by Mike Wood.
#Lon and Lat variables can be arrays, and if they are, they return an array.

def great_circle_distance(lon_ref, lat_ref, Lon, Lat):
    earth_radius = 6371000
    lon_ref_radians = np.radians(lon_ref)
    lat_ref_radians = np.radians(lat_ref)
    lons_radians = np.radians(Lon)
    lats_radians = np.radians(Lat)
    lat_diff = lats_radians - lat_ref_radians
    lon_diff = lons_radians - lon_ref_radians
    d = np.sin(lat_diff * 0.5) ** 2 + np.cos(lat_ref_radians) * np.cos(lats_radians) * np.sin(lon_diff * 0.5) ** 2
    h = 2 * earth_radius * np.arcsin(np.sqrt(d))
    return(h)

In [10]:
#Empty arrays to be added to dataframe.
blank_sediment_ID=[0]*len(sponge_df)
blank_sediment_type=[0]*len(sponge_df)
blank_sediment_supertype=[0]*len(sponge_df)

In [11]:
#lon and lat arrays made 2d in order to correspond to z values, which exist in a 2d array.
lon2d,lat2d=np.meshgrid(lon,lat)

#All raveled to be iterable.
lon_array=lon2d.ravel()
lat_array=lat2d.ravel()
z_array=z.ravel()

Unfortunately, the code created to match up sediment types takes a long time to run, due to the high number of sponges and density of the array. Code is provided below to estimate how long it would take for the function to run by testing with 100 sponges.

In [14]:

i=0 #used to match position in sponge dataframe to lithology arrays
start_time=time.time() #record start time
for index,sample in sponge_df.iterrows():
    if i<100: #limit to first 100 sponges
        #array of distance between raster points and sponge created
        dist_vector=great_circle_distance(sample.longitude, sample.latitude, lon_array, lat_array)
        z_index=np.argmin(dist_vector) #lowest distance recorded.
        sediment_ID=z_array[z_index] #index of lowest distance used to record sediment ID
        blank_sediment_ID[i]=sediment_ID #sediment ID recorded in array
        try:
            lithology=lithology_key[sediment_ID] #sediment ID recorded as sediment type
        except:
            lithology='No Data' #if sediment ID is invalid due to mask, recorded as no data
        blank_sediment_type[i]=lithology #sediment type recorded in array
        blank_sediment_supertype[i]=lithology_type_key[lithology] #sediment supertype also recorded
        
        i+=1 #increment i

time_for_100_entries=time.time() - start_time #record total duration in seconds
time_for_1_entry=time_for_100_entries/100 #estimate for 1 sponge
time_for_all_entries=time_for_1_entry*len(sponge_df.latitude) #estimate for all sponges
print("The program would take "+str(time_for_all_entries/(60*60))+" hours to run on your device under current conditions.")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
The program would take 81.14122030414806 hours to run on your device under current conditions.


The code below is the real version of the code to match the lithology to the sponges.

In [ ]:
i=0 #used to match position in sponge dataframe to lithology arrays

for index,sample in sponge_df.iterrows():
    #array of distance between raster points and sponge created
    dist_vector=great_circle_distance(sample.longitude, sample.latitude, lon_array, lat_array)
    z_index=np.argmin(dist_vector) #lowest distance recorded.
    sediment_ID=z_array[z_index] #index of lowest distance used to record sediment ID
    blank_sediment_ID[i]=sediment_ID #sediment ID recorded in array
    try:
        lithology=lithology_key[sediment_ID] #sediment ID recorded as sediment type
    except:
        lithology='No Data' #if sediment ID is invalid due to mask, recorded as no data
    blank_sediment_type[i]=lithology #sediment type recorded in array
    blank_sediment_supertype[i]=lithology_type_key[lithology] #sediment supertype also recorded
    
    #Print statements to see progress. Reports every 10 (if below 100), 100 (if below 1000), or 1000 sponges.
    i+=1
    if i<100:
        if i%10==0:
            print(i)
    elif i<1000:
        if i%100==0:
            print(i)
    elif i%1000==0:
        print(i)


sponge_df['sediment_ID']=blank_sediment_ID
sponge_df['sediment_type']=blank_sediment_type
sponge_df['sediment_supertype']=blank_sediment_supertype